In [ ]:
!nvidia-smi


In [ ]:
%pip install --upgrade pip
%pip install ultralytics
# If you ever hit cv2/libGL errors, also run:
# %pip install opencv-python-headless
# or (rarely needed):
# !apt-get update && apt-get install -y libgl1


In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11m-seg.pt')
print(model)


In [ ]:
import torch, platform
print("Torch CUDA:", torch.cuda.is_available(), torch.version.cuda, platform.platform())


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os, shutil, random, math
from pathlib import Path

# ==== CONFIG ====
BASE = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025")
DATASET = BASE / "dataset"         # where your mixed images+txt live
TRAIN   = BASE / "train"           # will be created/overwritten
VAL     = BASE / "val"             # will be created/overwritten
VAL_RATIO = 0.17                   # 17% for validation
RANDOM_SEED = 42                   # reproducible split
IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp"}  # case-insensitive
OVERWRITE = True                   # delete existing train/val before writing

# ==== PREP ====
if not DATASET.exists():
    raise FileNotFoundError(f"Dataset folder not found: {DATASET}")

def clean_dir(d: Path):
    if d.exists():
        if OVERWRITE:
            shutil.rmtree(d)
        else:
            raise RuntimeError(f"{d} exists. Set OVERWRITE=True or rename it first.")
    d.mkdir(parents=True, exist_ok=True)

clean_dir(TRAIN)
clean_dir(VAL)

# ==== COLLECT ELIGIBLE SAMPLES (image + matching .txt) ====
samples = []
for p in DATASET.rglob("*"):
    if p.is_file() and p.suffix.lower() in IMG_EXTS:
        txt = p.with_suffix(".txt")
        if txt.exists():
            samples.append((p, txt))

if not samples:
    raise RuntimeError(f"No (image, txt) pairs found under {DATASET}")

# ==== SHUFFLE & SPLIT ====
random.seed(RANDOM_SEED)
random.shuffle(samples)
n_total = len(samples)
n_val = math.floor(n_total * VAL_RATIO)
val_samples = samples[:n_val]
train_samples = samples[n_val:]

# ==== COPY ====
def copy_pair(img: Path, txt: Path, dst_dir: Path):
    # Flattens into train/ and val/ (same as your VM script)
    shutil.copy2(img, dst_dir / img.name)
    shutil.copy2(txt, dst_dir / txt.name)

for img, txt in val_samples:
    copy_pair(img, txt, VAL)
for img, txt in train_samples:
    copy_pair(img, txt, TRAIN)

# ==== REPORT ====
def count_images(d: Path):
    return sum(1 for f in d.glob("*") if f.suffix.lower() in IMG_EXTS)
def count_txts(d: Path):
    return sum(1 for f in d.glob("*.txt"))

print(f"Total pairs found: {n_total}")
print(f"Train pairs: {len(train_samples)} | images={count_images(TRAIN)} | labels={count_txts(TRAIN)}")
print(f"Val   pairs: {len(val_samples)}   | images={count_images(VAL)}   | labels={count_txts(VAL)}")

# ==== WRITE A YOLO DATA YAML ====
# Update class names to your exact mapping if needed.
yaml_text = f"""# Auto-generated for Colab
path: {BASE}          # project root
train: {TRAIN}        # images in this folder
val: {VAL}            # images in this folder

names:
  0: Crack
  1: ACrack
  2: Efflorescence
  3: WConccor
  4: Spalling
  5: Wetspot
  6: Rust
  7: ExposedRebars
"""
yaml_path = BASE / "data_colab.yaml"
yaml_path.write_text(yaml_text)
print(f"Wrote {yaml_path}")


In [ ]:
BASE="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025"
echo "Training images:";     find "$BASE/train" -iregex '.*\.\(jpg\|jpeg\|png\|webp\)' | wc -l
echo "Validation images:";   find "$BASE/val"   -iregex '.*\.\(jpg\|jpeg\|png\|webp\)' | wc -l


In [ ]:
BASE="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025"

!echo "Training images:";     find "$BASE/train" -iregex '.*\.\(jpg\|jpeg\|png\|webp\)' | wc -l
!echo "Validation images:";   find "$BASE/val"   -iregex '.*\.\(jpg\|jpeg\|png\|webp\)' | wc -l


In [ ]:
!yolo task=segment mode=train \
  model=yolo11m-seg.pt \
  data="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/data_colab.yaml" \
  epochs=250 imgsz=640 batch=-1 \
  workers=2 persistent_workers=False cache=True \
  lr0=0.001 lrf=0.01 momentum=0.937 weight_decay=0.0005 \
  patience=30 warmup_epochs=3 \
  degrees=30 translate=0.1 scale=0.5 shear=2 perspective=0.0005 \
  fliplr=0.5 flipud=0.3 hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 \
  project="/content/drive/MyDrive/yolo_runs" name="train_colab_m640"


In [ ]:
!yolo task=segment mode=train \
  model=yolo11m-seg.pt \
  data="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/data_colab.yaml" \
  epochs=250 imgsz=640 batch=-1 \
  workers=2 cache=True \
  lr0=0.001 lrf=0.01 momentum=0.937 weight_decay=0.0005 \
  patience=30 warmup_epochs=3 \
  degrees=30 translate=0.1 scale=0.5 shear=2 perspective=0.0005 \
  fliplr=0.5 flipud=0.3 hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 \
  project="/content/drive/MyDrive/yolo_runs" name="train_colab_m640"


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
!yolo predict \
  model="/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt" \
  source="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test" \
  conf=0.4 save=True


In [ ]:
!yolo predict \
  model="/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt" \
  source="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test" \
  conf=0.4 save=True \
  project="/content/drive/MyDrive/yolo_runs" name="trial_preds"


In [ ]:
from ultralytics import YOLO
from pathlib import Path

model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")

out_project = "/content/drive/MyDrive/yolo_runs"
out_name = "trial_preds_polygons"

results = model.predict(
    source="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test",
    imgsz=640,
    conf=0.4,
    save=True,
    max_det=300,
    project=out_project,
    name=out_name,
)

print("Saved to:", Path(out_project) / out_name)


In [ ]:
from ultralytics import YOLO
from pathlib import Path

model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")

out_project = "/content/drive/MyDrive/yolo_runs"
out_name = "trial_preds_polygons"

results = model.predict(
    source="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test",
    imgsz=640,
    conf=0.6,
    save=True,
    max_det=300,
    project=out_project,
    name=out_name,
)

print("Saved to:", Path(out_project) / out_name)


In [ ]:
from ultralytics import YOLO
from pathlib import Path

model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")

out_project = "/content/drive/MyDrive/yolo_runs"
out_name = "trial_preds_polygons"

results = model.predict(
    source="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test",
    imgsz=640,
    conf=0.5,
    save=True,
    max_det=300,
    project=out_project,
    name=out_name,
)

print("Saved to:", Path(out_project) / out_name)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
%pip install ultralytics --quiet


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
from ultralytics import YOLO
from pathlib import Path

model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")

out_project = "/content/drive/MyDrive/yolo_runs"
out_name = "trial_preds_polygons"

results = model.predict(
    source="/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test",
    imgsz=640,
    conf=0.5,
    save=True,
    max_det=300,
    project=out_project,
    name=out_name,
)

print("Saved to:", Path(out_project) / out_name)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet

In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ----- config -----
MODEL = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"
RUN_NAME = "trial_preds_polygons"  # results folder: {PROJECT}/{RUN_NAME}/
IMGSZ = 640
CONF = 0.5
BOX_ALPHA = 0.35  # transparency for the text box background

# ----- run prediction -----
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE, imgsz=IMGSZ, conf=CONF, save=True, max_det=300,
    project=PROJECT, name=RUN_NAME,
)

# ----- helper to draw a top-left counts box -----
def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        return
    # compute background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    # semi-transparent rectangle
    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    # text lines
    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    cv2.imwrite(str(img_path), img)

# ----- per-image counting + outputs -----
for res in results:
    save_dir = Path(getattr(res, "save_dir", PROJECT)) / RUN_NAME
    out_img = save_dir / Path(res.path).name  # annotated image path

    # collect class indices from predicted boxes (seg models still have boxes.cls)
    cls_idxs = []
    if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
        cls_tensor = res.boxes.cls
        cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

    counts = Counter(cls_idxs)
    lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

    # write a text file next to the annotated image
    txt_path = save_dir / f"{Path(res.path).stem}_counts.txt"
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    # overlay the counts box on the saved annotated image
    overlay_counts_on_image(out_img, lines)

print("Saved to:", Path(PROJECT) / RUN_NAME)
print("✅ Counts overlay added and *_counts.txt files written for each image.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"
RUN_NAME = "trial_preds_polygons"   # YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPER: draw top-left counts box on the saved image ======
def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        return
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    cv2.imwrite(str(img_path), img)

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
for res in results:
    save_dir = Path(res.save_dir)                             # the actual folder YOLO wrote to (e.g., .../trial_preds_polygons5/)
    out_img  = save_dir / Path(res.path).name                 # path to saved annotated image

    # collect class indices from predicted boxes (seg models also have boxes.cls)
    cls_idxs = []
    if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
        cls_tensor = res.boxes.cls
        cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

    counts = Counter(cls_idxs)
    lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

    # write per-image counts file next to the annotated image
    txt_path = save_dir / f"{Path(res.path).stem}_counts.txt"
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    # overlay counts box on the saved annotated image
    overlay_counts_on_image(out_img, lines)

print("✅ Done. Outputs are in:", Path(results[0].save_dir) if results else Path(PROJECT)/RUN_NAME)


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"
RUN_NAME = "trial_preds_polygons"   # YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Your trained model
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")

# Folder with your test videos
src_dir = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")

# Where to save annotated outputs (in Drive so they persist)
out_project = "/content/drive/MyDrive/yolo_runs"
out_name = "video_preds_polygons"   # results will go to /content/drive/MyDrive/yolo_runs/video_preds_polygons/

# Either let Ultralytics handle the whole folder at once...
results = model.predict(
    source=str(src_dir),   # processes all videos/images inside this folder
    imgsz=640,
    conf=0.5,
    save=True,
    max_det=300,
    project=out_project,
    name=out_name,
)

print("Saved to:", f"{out_project}/{out_name}")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter, defaultdict
import torch
import math

# ====== CONFIG ======
MODEL     = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
VIDEO_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT   = "/content/drive/MyDrive/yolo_runs"
RUN_NAME  = "video_preds_polygons_tracked"   # YOLO may append a number if exists
IMGSZ     = 640
CONF      = 0.5
TRACKER   = "bytetrack.yaml"  # robust default tracker (ships with Ultralytics)

# Process only typical video extensions
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ====== LOAD MODEL ======
model = YOLO(MODEL)

# ====== LOOP VIDEOS ======
videos = [p for p in VIDEO_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    print(f"No videos found in {VIDEO_DIR}")
else:
    print(f"Found {len(videos)} video(s) in {VIDEO_DIR}")

for vf in videos:
    print(f"\n▶ Tracking: {vf.name}")
    # Run tracker (returns per-frame Results)
    vid_results = model.track(
        source=str(vf),
        imgsz=IMGSZ,
        conf=CONF,
        save=True,
        tracker=TRACKER,
        project=PROJECT,
        name=RUN_NAME,   # YOLO may auto-append a number; we’ll read save_dir below
        verbose=False
    )

    if not vid_results:
        print(f"  ⚠️ No results returned for {vf.name}")
        continue

    # Where outputs were saved for this video
    save_dir = Path(vid_results[0].save_dir)
    names = vid_results[0].names  # class names mapping

    # === Count unique tracks per class over the entire video ===
    # For each class, keep a set of track IDs (stable across frames)
    unique_ids_per_class = defaultdict(set)
    # Fallback counter if no IDs are produced (rare, but handle it)
    fallback_new_object_counts = Counter()

    had_ids = False

    for res in vid_results:
        # If tracker assigned IDs, we get them in res.boxes.id (Tensor of shape [N])
        if getattr(res, "boxes", None) is not None:
            cls_t = getattr(res.boxes, "cls", None)
            id_t  = getattr(res.boxes, "id", None)  # <-- present when tracking works

            if id_t is not None:
                had_ids = True
                # Move to CPU lists
                cls_list = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
                id_list  = id_t.detach().cpu().tolist()  if isinstance(id_t, torch.Tensor)  else list(id_t or [])

                for c, tid in zip(cls_list, id_list):
                    # Some trackers can output -1 or NaN before stable assignment; ignore those
                    if tid is None or (isinstance(tid, float) and (math.isnan(tid) or tid < 0)) or (isinstance(tid, (int, float)) and tid < 0):
                        continue
                    c = int(c)
                    tid = int(tid)
                    unique_ids_per_class[c].add(tid)
            else:
                # Fallback if tracker didn't attach IDs: very conservative
                # (Counts per frame, but we will dedupe poorly; recommend using IDs)
                if cls_t is not None:
                    cls_list = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
                    for c in cls_list:
                        fallback_new_object_counts[int(c)] += 1

    # Build lines for output
    if had_ids:
        # Count unique track IDs per class → the robust, non-duplicated counts
        lines = [f"{names[c]}: {len(ids)}" for c, ids in sorted(unique_ids_per_class.items()) if len(ids) > 0]
        if not lines:
            lines = ["No detections"]
    else:
        # Fallback if tracker IDs weren’t produced (shouldn’t happen with bytetrack)
        lines = [f"{names[c]}: {fallback_new_object_counts[c]}" for c in sorted(fallback_new_object_counts.keys())] or ["No detections"]
        lines.insert(0, "(fallback: tracker IDs unavailable)")

    # Write one summary counts file per video, next to the saved video
    txt_path = save_dir / f"{vf.stem}_counts.txt"
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Tracked video outputs and *_counts.txt files are inside:")
print(Path(PROJECT) / RUN_NAME, "(or with an auto-appended number if re-run)")


In [ ]:
%pip install -q "lap>=0.5.12"


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import torch, math

# ==== CONFIG ====
MODEL     = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
VIDEO_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT   = "/content/drive/MyDrive/yolo_runs"
RUN_NAME  = "video_preds_polygons_tracked"  # YOLO may append a number if it exists
IMGSZ     = 640
CONF      = 0.5
TRACKER   = "bytetrack.yaml"  # robust default

VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ==== LOAD MODEL ====
model = YOLO(MODEL)

videos = [p for p in VIDEO_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
print(f"Found {len(videos)} video(s) in {VIDEO_DIR}")

for vf in videos:
    print(f"\n▶ Tracking: {vf.name}")

    # Collect unique track IDs per class over the entire video
    unique_ids_per_class = defaultdict(set)
    names = None
    save_dir = None
    had_ids = False

    # stream=True avoids storing all frames in memory
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ,
        conf=CONF,
        tracker=TRACKER,
        save=True,
        project=PROJECT,
        name=RUN_NAME,
        stream=True,
        verbose=False,
    ):
        if save_dir is None:
            save_dir = Path(res.save_dir)
            names = res.names

        if getattr(res, "boxes", None) is None:
            continue

        cls_t = getattr(res.boxes, "cls", None)
        id_t  = getattr(res.boxes, "id",  None)  # present when tracker is active

        if id_t is None or cls_t is None:
            continue

        had_ids = True

        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            # ignore invalid IDs
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

    if save_dir is None:
        print("  ⚠️ No output save_dir returned; skipping counts file.")
        continue

    # Build lines (unique track counts per class)
    if had_ids:
        lines = [f"{names[c]}: {len(ids)}" for c, ids in sorted(unique_ids_per_class.items()) if len(ids) > 0]
        if not lines:
            lines = ["No detections"]
    else:
        lines = ["(fallback) No tracker IDs produced", "No detections"]

    # Write summary counts next to the saved video
    txt_path = save_dir / f"{vf.stem}_counts.txt"
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Outputs (videos + *_counts.txt) are in:", Path(PROJECT) / RUN_NAME, "(or with an auto-appended number)")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import torch, math

# ==== CONFIG ====
MODEL     = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
VIDEO_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT   = "/content/drive/MyDrive/yolo_runs"
RUN_NAME  = "video_preds_polygons_tracked"  # YOLO may append a number if it exists
IMGSZ     = 640
CONF      = 0.5
TRACKER   = "bytetrack.yaml"  # robust default

VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ==== LOAD MODEL ====
model = YOLO(MODEL)

videos = [p for p in VIDEO_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
print(f"Found {len(videos)} video(s) in {VIDEO_DIR}")

for vf in videos:
    print(f"\n▶ Tracking: {vf.name}")

    # Collect unique track IDs per class over the entire video
    unique_ids_per_class = defaultdict(set)
    names = None
    save_dir = None
    had_ids = False

    # stream=True avoids storing all frames in memory
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ,
        conf=CONF,
        tracker=TRACKER,
        save=True,
        project=PROJECT,
        name=RUN_NAME,
        stream=True,
        verbose=False,
    ):
        if save_dir is None:
            save_dir = Path(res.save_dir)
            names = res.names

        if getattr(res, "boxes", None) is None:
            continue

        cls_t = getattr(res.boxes, "cls", None)
        id_t  = getattr(res.boxes, "id",  None)  # present when tracker is active

        if id_t is None or cls_t is None:
            continue

        had_ids = True

        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            # ignore invalid IDs
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

    if save_dir is None:
        print("  ⚠️ No output save_dir returned; skipping counts file.")
        continue

    # Build lines (unique track counts per class)
    if had_ids:
        lines = [f"{names[c]}: {len(ids)}" for c, ids in sorted(unique_ids_per_class.items()) if len(ids) > 0]
        if not lines:
            lines = ["No detections"]
    else:
        lines = ["(fallback) No tracker IDs produced", "No detections"]

    # Write summary counts next to the saved video
    txt_path = save_dir / f"{vf.stem}_counts.txt"
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Outputs (videos + *_counts.txt) are in:", Path(PROJECT) / RUN_NAME, "(or with an auto-appended number)")


In [ ]:
# If you still get 'lap' warnings, ensure it's installed once:
# %pip install -q "lap>=0.5.12"

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import shutil, os, math, torch

# ===== CONFIG =====
MODEL      = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
DRIVE_VIDS = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
LOCAL_VIDS = Path("/content/tmp_videos")        # process from local disk
LOCAL_OUT  = Path("/content/track_runs")        # write results locally
DRIVE_OUT  = Path("/content/drive/MyDrive/yolo_runs")  # final destination in Drive
RUN_NAME   = "video_preds_polygons_tracked"     # Ultralytics may append a number
IMGSZ      = 640
CONF       = 0.5
TRACKER    = "bytetrack.yaml"
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}
PRINT_EVERY = 25           # print a progress line every N frames
USE_FP16    = True         # half precision
VID_STRIDE  = 1            # set to 2 to skip every other frame (faster, slightly less accurate)

# ===== PREP LOCAL INPUT =====
LOCAL_VIDS.mkdir(parents=True, exist_ok=True)
# copy videos from Drive to local (overwrite if changed)
src_videos = [p for p in DRIVE_VIDS.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not src_videos:
    raise SystemExit(f"No videos found in {DRIVE_VIDS}")

for p in src_videos:
    dest = LOCAL_VIDS / p.name
    if not dest.exists():
        shutil.copy2(p, dest)

print(f"Processing {len(src_videos)} video(s) from {LOCAL_VIDS}")

# ===== RUN TRACKING PER VIDEO =====
model = YOLO(MODEL)

for vf in src_videos:
    local_vf = LOCAL_VIDS / vf.name
    print(f"\n▶ Tracking: {local_vf.name}")

    unique_ids_per_class = defaultdict(set)
    names = None
    save_dir = None
    had_ids = False
    frame_idx = 0

    # stream=True: yields one Results per frame; verbose=True prints Ultralytics logs
    for res in model.track(
        source=str(local_vf),
        device=0,
        imgsz=IMGSZ,
        conf=CONF,
        tracker=TRACKER,
        save=True,
        project=str(LOCAL_OUT),
        name=RUN_NAME,
        stream=True,
        verbose=True,
        half=USE_FP16,
        vid_stride=VID_STRIDE,
    ):
        # first frame: capture output dir and names
        if save_dir is None:
            save_dir = Path(res.save_dir)
            names = res.names
            print(f"Output will be in: {save_dir}")

        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  • processed frame {frame_idx}")

        if getattr(res, "boxes", None) is None:
            continue
        cls_t = getattr(res.boxes, "cls", None)
        id_t  = getattr(res.boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None or (isinstance(tid, float) and (math.isnan(tid) or tid < 0)) or (isinstance(tid, (int, float)) and tid < 0):
                continue
            unique_ids_per_class[int(c)].add(int(tid))

    if save_dir is None:
        print("  ⚠️ No frames produced; skipping.")
        continue

    # build counts text (unique track IDs per class)
    if had_ids:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
        if not lines:
            lines = ["No detections"]
    else:
        lines = ["(fallback) No tracker IDs produced", "No detections"]

    txt_path = save_dir / f"{vf.stem}_counts.txt"
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

# ===== MOVE RESULT FOLDER BACK TO DRIVE =====
# Find the actual run folder (Ultralytics may have appended a number)
made = [p for p in LOCAL_OUT.iterdir() if p.is_dir() and p.name.startswith(RUN_NAME)]
for folder in made:
    dest = DRIVE_OUT / folder.name
    # move or merge into Drive
    if dest.exists():
        # if already exists, append a suffix to avoid overwrite
        i = 2
        while (DRIVE_OUT / f"{folder.name}_{i}").exists():
            i += 1
        dest = DRIVE_OUT / f"{folder.name}_{i}"
    shutil.move(str(folder), str(dest))
    print(f"📦 Moved results to Drive: {dest}")

print("\n🎬 Done.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Your trained model
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")

# Folder with your test videos
src_dir = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")

# Where to save annotated outputs (in Drive so they persist)
out_project = "/content/drive/MyDrive/yolo_runs"
out_name = "video_preds_polygons"   # results will go to /content/drive/MyDrive/yolo_runs/video_preds_polygons/

# Either let Ultralytics handle the whole folder at once...
results = model.predict(
    source=str(src_dir),   # processes all videos/images inside this folder
    imgsz=640,
    conf=0.5,
    save=True,
    max_det=300,
    project=out_project,
    name=out_name,
)

print("Saved to:", f"{out_project}/{out_name}")


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
# If you still see a 'lap' warning once, run:
# %pip install -q "lap>=0.5.12"

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# === CONFIG ===
MODEL     = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
VIDEO_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT   = "/content/drive/MyDrive/yolo_runs"           # only for placing the *_counts.txt
RUN_NAME  = "video_counts_only_tracked"                  # folder will be created under PROJECT
IMGSZ     = 512                                          # smaller = less compute
CONF      = 0.5
TRACKER   = "bytetrack.yaml"
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}
USE_FP16   = True
VID_STRIDE = 2                                           # process every 2nd frame (2x faster)
PRINT_EVERY = 50

model = YOLO(MODEL)
videos = [p for p in VIDEO_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
print(f"Found {len(videos)} video(s) in {VIDEO_DIR}")

for vf in videos:
    print(f"\n▶ Counting (no video saved): {vf.name}")

    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # <— do NOT save annotated frames
        stream=True,                # <— generator, no big buffering
        stream_buffer=False,        # <— don’t accumulate past results
        half=USE_FP16,
        vid_stride=VID_STRIDE,
        verbose=False,
        project=PROJECT,            # needed only so we can place a folder for counts
        name=RUN_NAME,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # write counts file to a stable folder
    out_dir = Path(PROJECT) / RUN_NAME
    out_dir.mkdir(parents=True, exist_ok=True)
    txt_path = out_dir / f"{vf.stem}_counts.txt"

    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]

    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Counts are in:", Path(PROJECT) / RUN_NAME)


In [ ]:
# --- ONE-CELL: unique-ID counts per video + save annotated videos to the same folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL      = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR    = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT    = "/content/drive/MyDrive/yolo_runs"      # parent folder
RUN_NAME   = "video_preds_polygons"                  # <— both counts and videos go here
IMGSZ_CNT  = 512                                     # counting pass (lighter)
IMGSZ_VID  = 640                                     # saved video pass
CONF       = 0.5
TRACKER    = "bytetrack.yaml"
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}
USE_FP16   = True
VID_STRIDE_CNT = 2                                   # faster counting
VID_STRIDE_VID = 1                                   # best quality for saved videos
PRINT_EVERY = 50

# Ensure source videos exist
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# Create the shared output dir (counts + videos will live here)
run_dir = Path(PROJECT) / RUN_NAME
run_dir.mkdir(parents=True, exist_ok=True)

# ================= PASS 1: COUNT UNIQUE IDS (RAM-SAFE, NO VIDEO SAVED) =================
model = YOLO(MODEL)
print(f"Counting unique defects (no video saving) for {len(videos)} video(s)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                # <— no annotated frames saved
        stream=True,               # generator
        stream_buffer=False,       # don’t buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue
        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # write per-video counts file into the shared run folder
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path)
    for L in lines: print("   ", L)

# ================= PASS 2: SAVE ANNOTATED VIDEOS (SAME FOLDER) =================
print(f"\nSaving annotated videos into the same folder: {run_dir}")
_ = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,      # <— same folder as counts
    exist_ok=True,      # <— reuse the folder, don’t make ..._2, ..._3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

print("\n✅ All done. Counts and videos are in:", run_dir)


In [ ]:
# --- ONE-CELL: unique-ID counts per video + save annotated videos to the same folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL      = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR    = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT    = "/content/drive/MyDrive/yolo_runs"      # parent folder
RUN_NAME   = "video_preds_polygons"                  # <— both counts and videos go here
IMGSZ_CNT  = 512                                     # counting pass (lighter)
IMGSZ_VID  = 640                                     # saved video pass
CONF       = 0.5
TRACKER    = "bytetrack.yaml"
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}
USE_FP16   = True
VID_STRIDE_CNT = 2                                   # faster counting
VID_STRIDE_VID = 1                                   # best quality for saved videos
PRINT_EVERY = 50

# Ensure source videos exist
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# Create the shared output dir (counts + videos will live here)
run_dir = Path(PROJECT) / RUN_NAME
run_dir.mkdir(parents=True, exist_ok=True)

# ================= PASS 1: COUNT UNIQUE IDS (RAM-SAFE, NO VIDEO SAVED) =================
model = YOLO(MODEL)
print(f"Counting unique defects (no video saving) for {len(videos)} video(s)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                # <— no annotated frames saved
        stream=True,               # generator
        stream_buffer=False,       # don’t buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue
        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # write per-video counts file into the shared run folder
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path)
    for L in lines: print("   ", L)

# ================= PASS 2: SAVE ANNOTATED VIDEOS (SAME FOLDER) =================
print(f"\nSaving annotated videos into the same folder: {run_dir}")
_ = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,      # <— same folder as counts
    exist_ok=True,      # <— reuse the folder, don’t make ..._2, ..._3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

print("\n✅ All done. Counts and videos are in:", run_dir)



In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 512
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet

In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# === ONE CELL: per-video save (local) + unique-ID counts + move to Drive (auto-numbered folders) ===
# Tip: if you ever see a 'lap' requirement warning, run once:  %pip install -q "lap>=0.5.12"

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import shutil, math, torch, gc

# ---------------- CONFIG ----------------
MODEL = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"

# Source videos in Drive
SRC_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# Where final results should live (in Drive)
DRIVE_PROJECT = Path("/content/drive/MyDrive/yolo_runs")
RUN_NAME = "video_preds_polygons"  # YOLO will auto-append numbers per video: ...,_2,_3,...

# Local temp folder to avoid Drive I/O during inference (MUCH more stable)
LOCAL_PROJECT = Path("/content/vid_tmp")

# Predict (video saving) settings — keep these modest to avoid RAM spikes
IMGSZ_VID = 640        # use 640 if you prefer higher quality and have headroom
VID_STRIDE_VID = 2     # 2x faster & lighter (process every other frame)
CONF = 0.5
USE_FP16 = True
RETINA_MASKS = False   # lighter mask drawing
VERBOSE_PRED = False   # don't spam per-frame logs

# Tracking (counting) settings — RAM-safe
IMGSZ_CNT = 640
VID_STRIDE_CNT = 2
TRACKER = "bytetrack.yaml"
PRINT_EVERY = 100      # progress heartbeat

# -------------- PREP --------------
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

LOCAL_PROJECT.mkdir(parents=True, exist_ok=True)
DRIVE_PROJECT.mkdir(parents=True, exist_ok=True)

model = YOLO(MODEL)

for vf in videos:
    print(f"\n==================== {vf.name} ====================")

    # ---------- PASS A: SAVE ANNOTATED VIDEO LOCALLY ----------
    print("• Saving annotated video locally...")
    preds = model.predict(
        source=str(vf),
        imgsz=IMGSZ_VID,
        conf=CONF,
        save=True,
        project=str(LOCAL_PROJECT),
        name=RUN_NAME,          # no exist_ok → YOLO auto-numbers per video
        vid_stride=VID_STRIDE_VID,
        half=USE_FP16,
        retina_masks=RETINA_MASKS,
        verbose=VERBOSE_PRED,
        device=0,
    )
    if not preds:
        print("  ⚠️ No prediction results; skipping counts and move.")
        continue

    # The auto-numbered folder for THIS video (e.g., /content/vid_tmp/video_preds_polygons3)
    out_dir_local = Path(preds[0].save_dir)
    print("  Local run dir:", out_dir_local)

    # ---------- PASS B: COUNT UNIQUE DEFECTS (NO VIDEO SAVED) ----------
    print("• Counting unique defects with tracking (no saving)...")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,              # do NOT save frames here
        stream=True,             # generator
        stream_buffer=False,     # don't hold old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"   processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue
        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy
        del res
        if frame_idx % 200 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write counts file into the SAME local run folder as the video
    txt_path_local = out_dir_local / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path_local, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path_local)
    for L in lines:
        print("   ", L)

    # ---------- MOVE THIS VIDEO'S FOLDER TO DRIVE ----------
    dest = DRIVE_PROJECT / out_dir_local.name  # keep the auto-numbered folder name
    if dest.exists():
        # extremely rare; if exists, add suffix
        i = 2
        while (DRIVE_PROJECT / f"{out_dir_local.name}_{i}").exists():
            i += 1
        dest = DRIVE_PROJECT / f"{out_dir_local.name}_{i}"

    shutil.move(str(out_dir_local), str(dest))
    print("📦 Moved results to Drive:", dest)

print("\n✅ Done. All per-video folders (annotated video + *_counts.txt) are under:", DRIVE_PROJECT)


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
# === ONE CELL: per-video save (local) + unique-ID counts + move to Drive (auto-numbered folders) ===
# Tip: if you ever see a 'lap' requirement warning, run once:  %pip install -q "lap>=0.5.12"

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import shutil, math, torch, gc

# ---------------- CONFIG ----------------
MODEL = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"

# Source videos in Drive
SRC_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# Where final results should live (in Drive)
DRIVE_PROJECT = Path("/content/drive/MyDrive/yolo_runs")
RUN_NAME = "video_preds_polygons"  # YOLO will auto-append numbers per video: ...,_2,_3,...

# Local temp folder to avoid Drive I/O during inference (MUCH more stable)
LOCAL_PROJECT = Path("/content/vid_tmp")

# Predict (video saving) settings — keep these modest to avoid RAM spikes
IMGSZ_VID = 640        # use 640 if you prefer higher quality and have headroom
VID_STRIDE_VID = 2     # 2x faster & lighter (process every other frame)
CONF = 0.5
USE_FP16 = True
RETINA_MASKS = False   # lighter mask drawing
VERBOSE_PRED = False   # don't spam per-frame logs

# Tracking (counting) settings — RAM-safe
IMGSZ_CNT = 640
VID_STRIDE_CNT = 2
TRACKER = "bytetrack.yaml"
PRINT_EVERY = 100      # progress heartbeat

# -------------- PREP --------------
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

LOCAL_PROJECT.mkdir(parents=True, exist_ok=True)
DRIVE_PROJECT.mkdir(parents=True, exist_ok=True)

model = YOLO(MODEL)

for vf in videos:
    print(f"\n==================== {vf.name} ====================")

    # ---------- PASS A: SAVE ANNOTATED VIDEO LOCALLY ----------
    print("• Saving annotated video locally...")
    preds = model.predict(
        source=str(vf),
        imgsz=IMGSZ_VID,
        conf=CONF,
        save=True,
        project=str(LOCAL_PROJECT),
        name=RUN_NAME,          # no exist_ok → YOLO auto-numbers per video
        vid_stride=VID_STRIDE_VID,
        half=USE_FP16,
        retina_masks=RETINA_MASKS,
        verbose=VERBOSE_PRED,
        device=0,
    )
    if not preds:
        print("  ⚠️ No prediction results; skipping counts and move.")
        continue

    # The auto-numbered folder for THIS video (e.g., /content/vid_tmp/video_preds_polygons3)
    out_dir_local = Path(preds[0].save_dir)
    print("  Local run dir:", out_dir_local)

    # ---------- PASS B: COUNT UNIQUE DEFECTS (NO VIDEO SAVED) ----------
    print("• Counting unique defects with tracking (no saving)...")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,              # do NOT save frames here
        stream=True,             # generator
        stream_buffer=False,     # don't hold old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"   processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue
        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy
        del res
        if frame_idx % 200 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write counts file into the SAME local run folder as the video
    txt_path_local = out_dir_local / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path_local, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path_local)
    for L in lines:
        print("   ", L)

    # ---------- MOVE THIS VIDEO'S FOLDER TO DRIVE ----------
    dest = DRIVE_PROJECT / out_dir_local.name  # keep the auto-numbered folder name
    if dest.exists():
        # extremely rare; if exists, add suffix
        i = 2
        while (DRIVE_PROJECT / f"{out_dir_local.name}_{i}").exists():
            i += 1
        dest = DRIVE_PROJECT / f"{out_dir_local.name}_{i}"

    shutil.move(str(out_dir_local), str(dest))
    print("📦 Moved results to Drive:", dest)

print("\n✅ Done. All per-video folders (annotated video + *_counts.txt) are under:", DRIVE_PROJECT)

In [ ]:
# === ONE CELL: per-video save (local, streamed) + unique-ID counts + move to Drive ===

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import shutil, math, torch, gc

# ---------------- CONFIG ----------------
MODEL = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"

SRC_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

DRIVE_PROJECT = Path("/content/drive/MyDrive/yolo_runs")
RUN_NAME = "video_preds_polygons"      # YOLO will auto-number per video

LOCAL_PROJECT = Path("/content/vid_tmp")  # local temp output

# Predict (video saving) settings
IMGSZ_VID = 640          # use 640 if you have headroom
VID_STRIDE_VID = 2
CONF = 0.5
USE_FP16 = True
RETINA_MASKS = False
PRINT_EVERY_VID = 100    # heartbeat during saving

# Tracking (counting) settings
IMGSZ_CNT = 640
VID_STRIDE_CNT = 2
TRACKER = "bytetrack.yaml"
PRINT_EVERY_CNT = 100

# -------------- PREP --------------
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

LOCAL_PROJECT.mkdir(parents=True, exist_ok=True)
DRIVE_PROJECT.mkdir(parents=True, exist_ok=True)

model = YOLO(MODEL)

for vf in videos:
    print(f"\n==================== {vf.name} ====================")

    # ---------- PASS A: SAVE ANNOTATED VIDEO LOCALLY (streamed, with progress) ----------
    print("• Saving annotated video locally (streaming, no RAM buildup)...")
    out_dir_local = None
    frame_idx = 0

    for res in model.predict(
        source=str(vf),
        imgsz=IMGSZ_VID,
        conf=CONF,
        save=True,
        project=str(LOCAL_PROJECT),
        name=RUN_NAME,           # auto-numbered folder will be created
        vid_stride=VID_STRIDE_VID,
        half=USE_FP16,
        retina_masks=RETINA_MASKS,
        verbose=False,
        device=0,
        stream=True,             # <<--- IMPORTANT: stream results, don't accumulate
    ):
        if out_dir_local is None:
            out_dir_local = Path(res.save_dir)   # e.g., /content/vid_tmp/video_preds_polygons3
            print("  Local run dir:", out_dir_local)
        frame_idx += 1
        if frame_idx % PRINT_EVERY_VID == 0:
            print(f"   saved frame {frame_idx}")
        # free per-frame object
        del res
        if frame_idx % 200 == 0:
            torch.cuda.empty_cache(); gc.collect()

    if out_dir_local is None:
        print("  ⚠️ No frames saved; skipping counts and move.")
        continue

    # ---------- PASS B: COUNT UNIQUE DEFECTS (NO VIDEO SAVED) ----------
    print("• Counting unique defects with tracking (no saving, streaming)...")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,               # do NOT save frames here
        stream=True,              # generator
        stream_buffer=False,      # don't hold old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY_CNT == 0:
            print(f"   processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue
        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        del res
        if frame_idx % 200 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write counts file into the SAME local run folder as the video
    txt_path_local = out_dir_local / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path_local, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path_local)
    for L in lines:
        print("   ", L)

    # ---------- MOVE THIS VIDEO'S FOLDER TO DRIVE ----------
    dest = DRIVE_PROJECT / out_dir_local.name  # keep auto-numbered folder name
    if dest.exists():
        i = 2
        while (DRIVE_PROJECT / f"{out_dir_local.name}_{i}").exists():
            i += 1
        dest = DRIVE_PROJECT / f"{out_dir_local.name}_{i}"
    shutil.move(str(out_dir_local), str(dest))
    print("📦 Moved results to Drive:", dest)

print("\n✅ Done. All per-video folders (annotated video + *_counts.txt) are under:", DRIVE_PROJECT)


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
# --- ONE CELL: save annotated videos AND write unique-ID counts, with auto-numbered run folder ---

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

# ================= CONFIG =================
MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"    # parent folder for runs
RUN_NAME    = "video_preds_polygons"                # base name (YOLO will append numbers automatically)
CONF        = 0.5

# Video saving pass (prettier output)
IMGSZ_VID   = 640
VID_STRIDE_VID = 1

# Counting pass (RAM-safe + faster)
IMGSZ_CNT   = 640
VID_STRIDE_CNT = 2
USE_FP16    = True
TRACKER     = "bytetrack.yaml"
PRINT_EVERY = 50
VIDEO_EXTS  = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

# ================= CHECK SOURCE =================
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

# ================= LOAD MODEL =================
model = YOLO(MODEL)

# ================= PASS 1: SAVE ANNOTATED VIDEOS =================
print("Saving annotated videos (YOLO will auto-append run folder number if needed)...")
pred_results = model.predict(
    source=str(SRC_DIR),
    imgsz=IMGSZ_VID,
    conf=CONF,
    save=True,
    project=PROJECT,
    name=RUN_NAME,     # <-- no exist_ok, so YOLO creates video_preds_polygons, ...2, ...3, etc.
    vid_stride=VID_STRIDE_VID,
    verbose=True,
)

# Get the actual auto-numbered folder YOLO just created
if not pred_results:
    raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir)
print(f"\n✅ Videos saved to: {run_dir}")

# ================= PASS 2: COUNT UNIQUE DEFECTS PER VIDEO =================
print("\nCounting unique defects (no video saving)...")

for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    # stream=True + save=False keeps RAM usage low
    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,                 # DO NOT save frames in counting pass
        stream=True,                # generator
        stream_buffer=False,        # don't buffer old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0:
            print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)  # tracker IDs present when tracking works
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        # keep memory tidy on long videos
        del res
        if frame_idx % 100 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write per-video counts file into the SAME run_dir as the videos
    txt_path = run_dir / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path, "w") as f:
        f.write("\n".join(lines) + "\n")

    print("  ✅ Wrote counts:", txt_path)
    for L in lines:
        print("   ", L)

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)


In [ ]:
# === ONE CELL: per-video save (local, streamed) + unique-ID counts + move to Drive ===

from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import shutil, math, torch, gc

# ---------------- CONFIG ----------------
MODEL = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"

SRC_DIR = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
VIDEO_EXTS = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

DRIVE_PROJECT = Path("/content/drive/MyDrive/yolo_runs")
RUN_NAME = "video_preds_polygons"      # YOLO will auto-number per video

LOCAL_PROJECT = Path("/content/vid_tmp")  # local temp output

# Predict (video saving) settings
IMGSZ_VID = 640          # use 640 if you have headroom
VID_STRIDE_VID = 2
CONF = 0.5
USE_FP16 = True
RETINA_MASKS = False
PRINT_EVERY_VID = 100    # heartbeat during saving

# Tracking (counting) settings
IMGSZ_CNT = 640
VID_STRIDE_CNT = 2
TRACKER = "bytetrack.yaml"
PRINT_EVERY_CNT = 100

# -------------- PREP --------------
videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos:
    raise SystemExit(f"No videos found in {SRC_DIR}")

LOCAL_PROJECT.mkdir(parents=True, exist_ok=True)
DRIVE_PROJECT.mkdir(parents=True, exist_ok=True)

model = YOLO(MODEL)

for vf in videos:
    print(f"\n==================== {vf.name} ====================")

    # ---------- PASS A: SAVE ANNOTATED VIDEO LOCALLY (streamed, with progress) ----------
    print("• Saving annotated video locally (streaming, no RAM buildup)...")
    out_dir_local = None
    frame_idx = 0

    for res in model.predict(
        source=str(vf),
        imgsz=IMGSZ_VID,
        conf=CONF,
        save=True,
        project=str(LOCAL_PROJECT),
        name=RUN_NAME,           # auto-numbered folder will be created
        vid_stride=VID_STRIDE_VID,
        half=USE_FP16,
        retina_masks=RETINA_MASKS,
        verbose=False,
        device=0,
        stream=True,             # <<--- IMPORTANT: stream results, don't accumulate
    ):
        if out_dir_local is None:
            out_dir_local = Path(res.save_dir)   # e.g., /content/vid_tmp/video_preds_polygons3
            print("  Local run dir:", out_dir_local)
        frame_idx += 1
        if frame_idx % PRINT_EVERY_VID == 0:
            print(f"   saved frame {frame_idx}")
        # free per-frame object
        del res
        if frame_idx % 200 == 0:
            torch.cuda.empty_cache(); gc.collect()

    if out_dir_local is None:
        print("  ⚠️ No frames saved; skipping counts and move.")
        continue

    # ---------- PASS B: COUNT UNIQUE DEFECTS (NO VIDEO SAVED) ----------
    print("• Counting unique defects with tracking (no saving, streaming)...")
    unique_ids_per_class = defaultdict(set)
    names = None
    had_ids = False
    frame_idx = 0

    for res in model.track(
        source=str(vf),
        imgsz=IMGSZ_CNT,
        conf=CONF,
        tracker=TRACKER,
        device=0,
        save=False,               # do NOT save frames here
        stream=True,              # generator
        stream_buffer=False,      # don't hold old frames
        half=USE_FP16,
        vid_stride=VID_STRIDE_CNT,
        verbose=False,
    ):
        if names is None:
            names = res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY_CNT == 0:
            print(f"   processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None:
            continue
        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None:
            continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])
        for c, tid in zip(clses, ids):
            if tid is None:
                continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0):
                continue
            if isinstance(tid, (int, float)) and tid < 0:
                continue
            unique_ids_per_class[int(c)].add(int(tid))

        del res
        if frame_idx % 200 == 0:
            torch.cuda.empty_cache(); gc.collect()

    # Write counts file into the SAME local run folder as the video
    txt_path_local = out_dir_local / f"{vf.stem}_counts.txt"
    if had_ids and unique_ids_per_class:
        lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s) > 0]
    else:
        lines = ["No detections"]
    with open(txt_path_local, "w") as f:
        f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path_local)
    for L in lines:
        print("   ", L)

    # ---------- MOVE THIS VIDEO'S FOLDER TO DRIVE ----------
    dest = DRIVE_PROJECT / out_dir_local.name  # keep auto-numbered folder name
    if dest.exists():
        i = 2
        while (DRIVE_PROJECT / f"{out_dir_local.name}_{i}").exists():
            i += 1
        dest = DRIVE_PROJECT / f"{out_dir_local.name}_{i}"
    shutil.move(str(out_dir_local), str(dest))
    print("📦 Moved results to Drive:", dest)

print("\n✅ Done. All per-video folders (annotated video + *_counts.txt) are under:", DRIVE_PROJECT)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
# ====== PREDICT (keep as you have it above) ======
# model = YOLO(MODEL)
# results = model.predict(
#     source=SOURCE, imgsz=IMGSZ, conf=CONF, save=True, max_det=300,
#     project=PROJECT, name=RUN_NAME,
# )

# ====== JSON + CROPS + COUNTS + OVERLAY (REPLACE your old per-image loop with this) ======
import os, json, math, numpy as np, cv2, torch
from pathlib import Path
from collections import Counter

SAVE_JSON = True
SAVE_CROPS = True          # set False if you don't want crops
PAD_PX = 8                 # padding around crop
BOX_ALPHA = 0.35           # counts box transparency

def to_int_list(x): return [int(round(v)) for v in x]

def poly_area_px2(points_xy):
    if len(points_xy) < 3: return 0.0
    pts = np.asarray(points_xy, dtype=np.float32)
    return float(cv2.contourArea(pts))

def clamp_bbox(x1, y1, x2, y2, w, h):
    x1 = max(0, min(int(math.floor(x1)), w - 1))
    y1 = max(0, min(int(math.floor(y1)), h - 1))
    x2 = max(0, min(int(math.ceil(x2)),  w - 1))
    y2 = max(0, min(int(math.ceil(y2)),  h - 1))
    if x2 <= x1: x2 = min(x1 + 1, w - 1)
    if y2 <= y1: y2 = min(y1 + 1, h - 1)
    return x1, y1, x2, y2

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5
    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)
    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h
    return cv2.imwrite(str(img_path), img)

def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    direct = save_dir / src_path.name
    if direct.exists(): return direct
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # e.g., .../trial_preds_polygons5/
    json_dir  = final_dir / "json"
    crops_dir = final_dir / "crops"
    if SAVE_JSON:  json_dir.mkdir(parents=True, exist_ok=True)
    if SAVE_CROPS: crops_dir.mkdir(parents=True, exist_ok=True)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)
        src_path = Path(res.path)
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        names = res.names
        dets, cls_idxs, confs, bboxes = [], [], [], []

        if getattr(res, "boxes", None) is not None:
            if getattr(res.boxes, "cls", None) is not None:
                cls_raw = res.boxes.cls
                cls_idxs = (cls_raw.detach().cpu().tolist()
                            if isinstance(cls_raw, torch.Tensor) else list(cls_raw))
                cls_idxs = [int(x) for x in cls_idxs]
            if getattr(res.boxes, "conf", None) is not None:
                conf_raw = res.boxes.conf
                confs = (conf_raw.detach().cpu().tolist()
                         if isinstance(conf_raw, torch.Tensor) else list(conf_raw))
            if getattr(res.boxes, "xyxy", None) is not None:
                xyxy = res.boxes.xyxy
                if isinstance(xyxy, torch.Tensor):
                    xyxy = xyxy.detach().cpu().numpy()
                bboxes = xyxy.tolist()

        polys_xy = None
        if getattr(res, "masks", None) is not None and getattr(res.masks, "xy", None) is not None:
            polys_xy = res.masks.xy  # list of (Kx2) arrays in image coords

        N = len(polys_xy) if polys_xy is not None else len(bboxes)
        img_bgr = cv2.imread(str(src_path))
        ih, iw = (img_bgr.shape[0], img_bgr.shape[1]) if img_bgr is not None else (None, None)

        for i in range(N):
            cls_name = names[cls_idxs[i]] if i < len(cls_idxs) else names[0]
            conf     = float(confs[i]) if i < len(confs) else None

            if i < len(bboxes):
                x1, y1, x2, y2 = bboxes[i]
            else:
                if polys_xy is not None and i < len(polys_xy) and len(polys_xy[i]) >= 3:
                    px, py = polys_xy[i][:,0], polys_xy[i][:,1]
                    x1, y1, x2, y2 = float(px.min()), float(py.min()), float(px.max()), float(py.max())
                else:
                    x1 = y1 = 0.0
                    x2 = float(iw - 1) if iw else 1.0
                    y2 = float(ih - 1) if ih else 1.0
            bbox = [int(round(x1)), int(round(y1)), int(round(x2)), int(round(y2))]

            poly_flat, area_px2 = [], None
            if polys_xy is not None and i < len(polys_xy) and len(polys_xy[i]) >= 3:
                pts = polys_xy[i]
                area_px2 = poly_area_px2(pts)
                poly_flat = to_int_list(pts.reshape(-1).tolist())
            else:
                if iw is not None and ih is not None:
                    area_px2 = max(1, (bbox[2]-bbox[0])*(bbox[3]-bbox[1]))

            crop_rel_path = None
            if SAVE_CROPS and img_bgr is not None:
                x1p, y1p, x2p, y2p = clamp_bbox(bbox[0]-PAD_PX, bbox[1]-PAD_PX, bbox[2]+PAD_PX, bbox[3]+PAD_PX, iw, ih)
                crop = img_bgr[y1p:y2p, x1p:x2p]
                if crop.size > 0:
                    cls_dir = crops_dir / cls_name
                    cls_dir.mkdir(parents=True, exist_ok=True)
                    crop_name = f"{src_path.stem}_{cls_name}_{i:02d}.jpg"
                    cv2.imwrite(str(cls_dir / crop_name), crop)
                    crop_rel_path = str(Path("crops") / cls_name / crop_name)

            dets.append({
                "class": cls_name,
                "bbox_xyxy": bbox,
                "poly_xy_flat": poly_flat,
                "area_px2": area_px2,
                "conf": conf,
                "crop_relpath": crop_rel_path
            })

        if SAVE_JSON:
            rec = {"image_path": str(src_path), "save_path": str(out_img) if out_img else None, "detections": dets}
            (final_dir / "json").mkdir(exist_ok=True)
            with open(final_dir / "json" / f"{src_path.stem}.json", "w") as f:
                json.dump(rec, f, indent=2)

        counts = Counter([d["class"] for d in dets]) if dets else {}
        lines = [f"{k}: {counts[k]}" for k in sorted(counts.keys())] if counts else ["No detections"]

        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        if out_img is not None:
            if overlay_counts_on_image(out_img, lines):
                print(f"✅ JSON/Crops/Counts written for {out_img.name}")
            else:
                print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done: JSON at", final_dir / "json", "| Crops at", final_dir / "crops" if SAVE_CROPS else "(skipped)")


In [ ]:
# =========================
# YOLOv11 IMAGE PRED + JSON/CROPS EXPORT (Colab-ready)
# =========================

# --- Install (if needed) ---
# !pip install ultralytics opencv-python-headless==4.10.0.84

import os, json, math, glob
from pathlib import Path
from collections import Counter

import cv2
import numpy as np
import torch
import torch.nn.functional as F

from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# =========================
# CONFIG: EDIT THESE
# =========================
MODEL    = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE   = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT  = "/content/drive/MyDrive/yolo_runs"  # where YOLO saves runs
RUN_NAME = "trial_preds_polygons"              # run folder name (YOLO may append a suffix)
IMGSZ    = 640
CONF     = 0.5
MAX_DET  = 300

# Exports
SAVE_JSON   = True
SAVE_CROPS  = True
PAD_PX      = 8        # bbox padding for crops
BOX_ALPHA   = 0.35     # overlay transparency for the counts box

# =========================
# PATCH: Polygon-only overlays (no boxes, no shaded masks, no confidences)
# =========================
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop

__orig_plot = Results.plot
def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))
    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled.")

# =========================
# HELPERS
# =========================
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

def to_int_list(x):
    return [int(round(v)) for v in x]

def poly_area_px2(points_xy):
    if len(points_xy) < 3:
        return 0.0
    pts = np.asarray(points_xy, dtype=np.float32)
    return float(cv2.contourArea(pts))

def clamp_bbox(x1, y1, x2, y2, w, h):
    x1 = max(0, min(int(math.floor(x1)), w - 1))
    y1 = max(0, min(int(math.floor(y1)), h - 1))
    x2 = max(0, min(int(math.ceil(x2)),  w - 1))
    y2 = max(0, min(int(math.ceil(y2)),  h - 1))
    if x2 <= x1: x2 = min(x1 + 1, w - 1)
    if y2 <= y1: y2 = min(y1 + 1, h - 1)
    return x1, y1, x2, y2

# =========================
# PREDICT
# =========================
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=MAX_DET,
    project=PROJECT,
    name=RUN_NAME,
)

# =========================
# POSTPROCESS: JSON + CROPS + COUNTS OVERLAY
# =========================
import math  # used in clamp_bbox

if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)   # e.g., .../trial_preds_polygons5/
    json_dir  = final_dir / "json"
    crops_dir = final_dir / "crops"

    if SAVE_JSON:
        json_dir.mkdir(parents=True, exist_ok=True)
    if SAVE_CROPS:
        crops_dir.mkdir(parents=True, exist_ok=True)

    print("Writing outputs in:", final_dir)

    total_images = 0
    total_dets = 0

    for res in results:
        save_dir = Path(res.save_dir)
        src_path = Path(res.path)
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        names = res.names
        dets, cls_idxs, confs, bboxes = [], [], [], []

        # boxes
        if getattr(res, "boxes", None) is not None:
            if getattr(res.boxes, "cls", None) is not None:
                cls_raw = res.boxes.cls
                cls_idxs = (cls_raw.detach().cpu().tolist()
                            if isinstance(cls_raw, torch.Tensor) else list(cls_raw))
                cls_idxs = [int(x) for x in cls_idxs]
            if getattr(res.boxes, "conf", None) is not None:
                conf_raw = res.boxes.conf
                confs = (conf_raw.detach().cpu().tolist()
                         if isinstance(conf_raw, torch.Tensor) else list(conf_raw))
            if getattr(res.boxes, "xyxy", None) is not None:
                xyxy = res.boxes.xyxy
                if isinstance(xyxy, torch.Tensor):
                    xyxy = xyxy.detach().cpu().numpy()
                bboxes = xyxy.tolist()

        # polygons (image coords)
        polys_xy = None
        if getattr(res, "masks", None) is not None and getattr(res.masks, "xy", None) is not None:
            polys_xy = res.masks.xy  # list of (Kx2) arrays

        N = len(polys_xy) if polys_xy is not None else len(bboxes)

        img_bgr = cv2.imread(str(src_path))
        ih, iw = (img_bgr.shape[0], img_bgr.shape[1]) if img_bgr is not None else (None, None)

        for i in range(N):
            cls_name = names[cls_idxs[i]] if i < len(cls_idxs) else names[0]
            conf     = float(confs[i]) if i < len(confs) else None

            if i < len(bboxes):
                x1, y1, x2, y2 = bboxes[i]
            else:
                if polys_xy is not None and i < len(polys_xy) and len(polys_xy[i]) >= 3:
                    px, py = polys_xy[i][:, 0], polys_xy[i][:, 1]
                    x1, y1, x2, y2 = float(px.min()), float(py.min()), float(px.max()), float(py.max())
                else:
                    x1 = y1 = 0.0
                    x2 = float(iw - 1) if iw else 1.0
                    y2 = float(ih - 1) if ih else 1.0

            bbox = [int(round(x1)), int(round(y1)), int(round(x2)), int(round(y2))]

            # polygon + area
            poly_flat, area_px2 = [], None
            if polys_xy is not None and i < len(polys_xy) and len(polys_xy[i]) >= 3:
                pts = polys_xy[i]
                area_px2 = poly_area_px2(pts)
                poly_flat = to_int_list(pts.reshape(-1).tolist())
            else:
                if iw is not None and ih is not None:
                    area_px2 = max(1, (bbox[2]-bbox[0])*(bbox[3]-bbox[1]))

            # crop (optional)
            crop_rel_path = None
            if SAVE_CROPS and img_bgr is not None:
                x1p, y1p, x2p, y2p = clamp_bbox(bbox[0]-PAD_PX, bbox[1]-PAD_PX, bbox[2]+PAD_PX, bbox[3]+PAD_PX, iw, ih)
                crop = img_bgr[y1p:y2p, x1p:x2p]
                if crop.size > 0:
                    cls_dir = crops_dir / cls_name
                    cls_dir.mkdir(parents=True, exist_ok=True)
                    crop_name = f"{src_path.stem}_{cls_name}_{i:02d}.jpg"
                    cv2.imwrite(str(cls_dir / crop_name), crop)
                    crop_rel_path = str(Path("crops") / cls_name / crop_name)

            dets.append({
                "class": cls_name,
                "bbox_xyxy": bbox,
                "poly_xy_flat": poly_flat,
                "area_px2": area_px2,
                "conf": conf,
                "crop_relpath": crop_rel_path
            })

        # JSON
        if SAVE_JSON:
            rec = {"image_path": str(src_path), "save_path": str(out_img) if out_img else None, "detections": dets}
            with open(json_dir / f"{src_path.stem}.json", "w") as f:
                json.dump(rec, f, indent=2)

        # counts + overlay
        counts = Counter([d["class"] for d in dets]) if dets else {}
        lines = [f"{k}: {counts[k]}" for k in sorted(counts.keys())] if counts else ["No detections"]

        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        if out_img is not None:
            overlay_counts_on_image(out_img, lines)

        total_images += 1
        total_dets   += len(dets)

        print(f"✅ {src_path.name}: {len(dets)} detections | counts ->", ", ".join(lines))

    print("\n🎉 Done")
    if SAVE_JSON:  print("   JSON dir :", json_dir)
    if SAVE_CROPS: print("   Crops dir:", crops_dir)
    print(f"   Images processed: {total_images} | Total detections: {total_dets}")


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
# =========================
# KEEP PREDICT EXACTLY AS YOU WROTE IT
# =========================
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch
import os, json, math, numpy as np  # extra imports for the new block

# ====== CONFIG (exactly yours) ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test 2"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # results root
RUN_NAME = "trial_preds_polygons"              # YOLO may append a number
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background (used below)

# ====== PREDICT (unchanged) ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# =========================
# NEW: JSON + CROPS + COUNTS + OVERLAY  (replaces your old per-image loop)
# =========================

# toggles
SAVE_JSON  = True
SAVE_CROPS = True
PAD_PX     = 8   # small padding around bbox for crops

def to_int_list(x):
    return [int(round(v)) for v in x]

def poly_area_px2(points_xy):
    """Compute polygon area in px^2 from Nx2 image-coord points."""
    if points_xy is None or len(points_xy) < 3:
        return 0.0
    pts = np.asarray(points_xy, dtype=np.float32)
    return float(cv2.contourArea(pts))

def clamp_bbox(x1, y1, x2, y2, w, h):
    x1 = max(0, min(int(math.floor(x1)), w - 1))
    y1 = max(0, min(int(math.floor(y1)), h - 1))
    x2 = max(0, min(int(math.ceil(x2)),  w - 1))
    y2 = max(0, min(int(math.ceil(y2)),  h - 1))
    if x2 <= x1: x2 = min(x1 + 1, w - 1)
    if y2 <= y1: y2 = min(y1 + 1, h - 1)
    return x1, y1, x2, y2

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5
    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)
    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h
    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # e.g., .../trial_preds_polygons5/
    json_dir  = final_dir / "json"
    crops_dir = final_dir / "crops"
    if SAVE_JSON:
        json_dir.mkdir(parents=True, exist_ok=True)
    if SAVE_CROPS:
        crops_dir.mkdir(parents=True, exist_ok=True)

    print("Writing outputs in:", final_dir)

    total_images = 0
    total_dets = 0

    for res in results:
        save_dir = Path(res.save_dir)
        src_path = Path(res.path)

        # 1) saved visualization produced by your polygon-only patch
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        names = res.names
        dets, cls_idxs, confs, bboxes = [], [], [], []

        # 2) boxes, classes, confidences
        if getattr(res, "boxes", None) is not None:
            if getattr(res.boxes, "cls", None) is not None:
                cls_raw = res.boxes.cls
                cls_idxs = (cls_raw.detach().cpu().tolist()
                            if isinstance(cls_raw, torch.Tensor) else list(cls_raw))
                cls_idxs = [int(x) for x in cls_idxs]
            if getattr(res.boxes, "conf", None) is not None:
                conf_raw = res.boxes.conf
                confs = (conf_raw.detach().cpu().tolist()
                         if isinstance(conf_raw, torch.Tensor) else list(conf_raw))
            if getattr(res.boxes, "xyxy", None) is not None:
                xyxy = res.boxes.xyxy
                if isinstance(xyxy, torch.Tensor):
                    xyxy = xyxy.detach().cpu().numpy()
                bboxes = xyxy.tolist()

        # 3) polygons in image coords (preferred for area)
        polys_xy = None
        if getattr(res, "masks", None) is not None and getattr(res.masks, "xy", None) is not None:
            polys_xy = res.masks.xy  # list of (Kx2) arrays

        # 4) iterate detections
        N = len(polys_xy) if polys_xy is not None else len(bboxes)

        img_bgr = cv2.imread(str(src_path))
        ih, iw = (img_bgr.shape[0], img_bgr.shape[1]) if img_bgr is not None else (None, None)

        for i in range(N):
            cls_name = names[cls_idxs[i]] if i < len(cls_idxs) else names[0]
            conf     = float(confs[i]) if i < len(confs) else None

            # bbox from boxes or from polygon bounds
            if i < len(bboxes):
                x1, y1, x2, y2 = bboxes[i]
            else:
                if polys_xy is not None and i < len(polys_xy) and len(polys_xy[i]) >= 3:
                    px, py = polys_xy[i][:, 0], polys_xy[i][:, 1]
                    x1, y1, x2, y2 = float(px.min()), float(py.min()), float(px.max()), float(py.max())
                else:
                    x1 = y1 = 0.0
                    x2 = float(iw - 1) if iw else 1.0
                    y2 = float(ih - 1) if ih else 1.0
            bbox = [int(round(x1)), int(round(y1)), int(round(x2)), int(round(y2))]

            # polygon + area
            poly_flat, area_px2 = [], None
            if polys_xy is not None and i < len(polys_xy) and len(polys_xy[i]) >= 3:
                pts = polys_xy[i]
                area_px2 = poly_area_px2(pts)
                poly_flat = to_int_list(pts.reshape(-1).tolist())
            else:
                if iw is not None and ih is not None:
                    area_px2 = max(1, (bbox[2]-bbox[0])*(bbox[3]-bbox[1]))

            # optional crop
            crop_rel_path = None
            if SAVE_CROPS and img_bgr is not None:
                x1p, y1p, x2p, y2p = clamp_bbox(bbox[0]-PAD_PX, bbox[1]-PAD_PX, bbox[2]+PAD_PX, bbox[3]+PAD_PX, iw, ih)
                crop = img_bgr[y1p:y2p, x1p:x2p]
                if crop.size > 0:
                    cls_dir = crops_dir / cls_name
                    cls_dir.mkdir(parents=True, exist_ok=True)
                    crop_name = f"{src_path.stem}_{cls_name}_{i:02d}.jpg"
                    cv2.imwrite(str(cls_dir / crop_name), crop)
                    crop_rel_path = str(Path("crops") / cls_name / crop_name)

            dets.append({
                "class": cls_name,
                "bbox_xyxy": bbox,
                "poly_xy_flat": poly_flat,   # [] if no mask
                "area_px2": area_px2,
                "conf": conf,
                "crop_relpath": crop_rel_path
            })

        # 5) JSON per image
        if SAVE_JSON:
            rec = {
                "image_path": str(src_path),
                "save_path": str(out_img) if out_img else None,
                "detections": dets
            }
            with open(json_dir / f"{src_path.stem}.json", "w") as f:
                json.dump(rec, f, indent=2)

        # 6) counts + overlay
        counts = Counter([d["class"] for d in dets]) if dets else {}
        lines = [f"{k}: {counts[k]}" for k in sorted(counts.keys())] if counts else ["No detections"]

        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        if out_img is not None:
            overlay_counts_on_image(out_img, lines)

        total_images += 1
        total_dets   += len(dets)
        print(f"✅ {src_path.name}: {len(dets)} detections | counts ->", ", ".join(lines))

    print("\n🎉 Done")
    if SAVE_JSON:  print("   JSON dir :", json_dir)
    if SAVE_CROPS: print("   Crops dir:", crops_dir)
    print(f"   Images processed: {total_images} | Total detections: {total_dets}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet

In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/images cowley county"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet

In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/images cowley county"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet

In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/images saline county"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/images sedgwick county"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%pip install ultralytics --quiet

In [ ]:
# === Polygon-only overlays for Ultralytics YOLO (no boxes, no shaded masks, no confidences) ===
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

# 1) Disable box labels (kills rectangles + conf text)
def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    return

# 2) Disable shaded masks
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False):
    return

# 3) Add polygon contour drawer with label (unshaded)
def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    """
    Draw polygon contours from a mask (tensor or uint8 array), no fill, with class label.
    """
    # Upsample + threshold if tensor
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)                       # [1,1,H,W]
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]),
                          mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask

    # Extract full-resolution contours
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3:
            continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

# Apply patches to the Annotator
Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

# 4) Patch Results.plot to call our polygon drawer instead of shaded masks/boxes
__orig_plot = Results.plot

def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    """
    Replacement for Results.plot:
    - no rectangles, no conf text, no filled masks
    - draws only polygon contours + class label
    """
    # Build fresh annotator image
    im = (self.orig_img.copy()
          if hasattr(self, "orig_img") and self.orig_img is not None
          else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))

    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)

    if pred_masks is not None and hasattr(pred_masks, "data") and pred_masks.data is not None:
        # Use class indices from boxes (if present), else default class 0
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))

    return annotator.result()

Results.plot = _plot_polygons_only

print("✅ Ultralytics patched: polygon-only overlays enabled (no boxes, no shaded masks, no confidences).")
print("   Restart runtime to undo the patch.")


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt")


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch

# ====== CONFIG ======
MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test"  # folder of test IMAGES
PROJECT = "/content/drive/MyDrive/yolo_runs"   # this is the folder where the results are saved
RUN_NAME = "trial_preds_polygons"   # and this is the name that will be given for each new folder where the predict will be stored YOLO may append a number if it already exists
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35  # transparency for the counts box background

# ====== PREDICT ======
model = YOLO(MODEL)
results = model.predict(
    source=SOURCE,
    imgsz=IMGSZ,
    conf=CONF,
    save=True,
    max_det=300,
    project=PROJECT,
    name=RUN_NAME,
)

# ====== HELPERS ======
def find_saved_image(save_dir: Path, src_path: Path) -> Path | None:
    """
    Find the actual visualization file YOLO wrote for a given source image.
    We try the straightforward path, then fall back to matching by stem.
    """
    direct = save_dir / src_path.name
    if direct.exists():
        return direct
    # fallback: same stem, any extension (handles JPG/PNG case, name tweaks)
    matches = list(save_dir.glob(src_path.stem + ".*"))
    return matches[0] if matches else None

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"⚠️ Could not read image for overlay: {img_path}")
        return False
    # background box size
    sizes = [cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w = max(sizes) + 20
    line_h = 28
    box_h = line_h * max(1, len(lines)) + 20
    x0, y0 = 5, 5

    overlay = img.copy()
    cv2.rectangle(overlay, (x0, y0), (x0 + box_w, y0 + box_h), (0, 0, 0), -1)
    img = cv2.addWeighted(overlay, box_alpha, img, 1 - box_alpha, 0)

    y = y0 + 20
    for t in lines:
        cv2.putText(img, t, (x0 + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                    (255, 255, 255), 2, cv2.LINE_AA)
        y += line_h

    ok = cv2.imwrite(str(img_path), img)
    if not ok:
        print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

# ====== PER-IMAGE COUNTS + SAVE TXT + OVERLAY ======
if not results:
    print("No results returned.")
else:
    final_dir = Path(results[0].save_dir)  # actual run folder (e.g., .../trial_preds_polygons5/)
    print("Writing outputs in:", final_dir)

    for res in results:
        save_dir = Path(res.save_dir)  # per-result (same as final_dir)
        src_path = Path(res.path)

        # 1) find the actual saved visualization for this source image
        out_img = find_saved_image(save_dir, src_path)
        if out_img is None:
            print(f"⚠️ Could not locate saved image for {src_path.name} in {save_dir}")
            continue

        # 2) collect predicted classes from boxes
        cls_idxs = []
        if getattr(res, "boxes", None) is not None and getattr(res.boxes, "cls", None) is not None:
            cls_tensor = res.boxes.cls
            cls_idxs = [int(x) for x in (cls_tensor.detach().cpu().tolist() if isinstance(cls_tensor, torch.Tensor) else cls_tensor)]

        counts = Counter(cls_idxs)
        lines = [f"{res.names[i]}: {counts[i]}" for i in sorted(counts.keys())] if counts else ["No detections"]

        # 3) write per-image counts text file
        txt_path = save_dir / f"{src_path.stem}_counts.txt"
        with open(txt_path, "w") as f:
            f.write("\n".join(lines) + "\n")

        # 4) overlay counts box on the saved visualization
        if overlay_counts_on_image(out_img, lines):
            print(f"✅ Overlay + counts written for {out_img.name}")
        else:
            print(f"⚠️ Skipped overlay for {out_img.name}")

    print("✅ Done.")


In [ ]:
# === EDIT ONLY THIS: put your existing repo name (the one that already has the VM code) ===
REPO_NAME   = "concrete-defect-yolov11"   # e.g., "yolo11-concrete-defects"
# ==========================================================================================

GITHUB_USER = "natnaeltaye"
BRANCH_NAME = "add-colab-workflow"  # feature branch we will PR into main

# Git identity
!git config --global user.name "natnaeltaye"
!git config --global user.email "myprecioushs@gmail.com"

print("Configured git:", !git config user.name, "/", !git config user.email)
print(f"Target repo: https://github.com/{GITHUB_USER}/{REPO_NAME}")
print(f"Branch to create: {BRANCH_NAME}")


In [ ]:
# === EDIT ONLY THIS: put your existing repo name (the one that already has the VM code) ===
REPO_NAME   = "concrete-defect-yolov11"
# ==========================================================================================

GITHUB_USER = "natnaeltaye"
BRANCH_NAME = "add-colab-workflow"  # feature branch we will PR into main

# Git identity
!git config --global user.name "natnaeltaye"
!git config --global user.email "myprecioushs@gmail.com"

# Show configured identity (run as separate shell commands)
print("Configured git:")
!git config user.name
!git config user.email

# These are normal Python prints – no shell "!" here
print(f"Target repo: https://github.com/{GITHUB_USER}/{REPO_NAME}")
print(f"Branch to create: {BRANCH_NAME}")


In [ ]:
# Create a classic GitHub PAT with 'repo' scope, then paste it below (input is hidden)
GITHUB_PAT = input("Paste your GitHub PAT (hidden): ").strip()
assert len(GITHUB_PAT) > 20, "Token looks too short. Generate a classic PAT with 'repo' scope."
print("✅ Token captured in memory.")


In [ ]:
import os, shutil, pathlib

REPO_NAME = "concrete-defect-yolov11"   # (already set before)
GITHUB_USER = "natnaeltaye"
BRANCH_NAME = "add-colab-workflow"

workdir = f"/content/{REPO_NAME}"
if pathlib.Path(workdir).exists():
    shutil.rmtree(workdir)

clone_url = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"
!git clone "$clone_url" "$workdir"
%cd "$workdir"

# Create feature branch
!git checkout -b "{BRANCH_NAME}"

# Prepare folder structure
!mkdir -p colab/scripts
print("✅ Cloned repo and created branch:", BRANCH_NAME)


In [ ]:
from pathlib import Path

content = r'''import os, shutil, random, math
from pathlib import Path

# ==== CONFIG ====
BASE = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025")
DATASET = BASE / "dataset"
TRAIN   = BASE / "train"
VAL     = BASE / "val"
VAL_RATIO = 0.17
RANDOM_SEED = 42
IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp"}
OVERWRITE = True

# ==== PREP ====
if not DATASET.exists():
    raise FileNotFoundError(f"Dataset folder not found: {DATASET}")

def clean_dir(d: Path):
    if d.exists():
        if OVERWRITE: shutil.rmtree(d)
        else: raise RuntimeError(f"{d} exists. Set OVERWRITE=True or rename it first.")
    d.mkdir(parents=True, exist_ok=True)

clean_dir(TRAIN); clean_dir(VAL)

# ==== COLLECT ELIGIBLE SAMPLES (image + matching .txt) ====
samples = []
for p in DATASET.rglob("*"):
    if p.is_file() and p.suffix.lower() in IMG_EXTS:
        txt = p.with_suffix(".txt")
        if txt.exists():
            samples.append((p, txt))

if not samples:
    raise RuntimeError(f"No (image, txt) pairs found under {DATASET}")

# ==== SHUFFLE & SPLIT ====
random.seed(RANDOM_SEED)
random.shuffle(samples)
n_total = len(samples)
n_val = math.floor(n_total * VAL_RATIO)
val_samples = samples[:n_val]
train_samples = samples[n_val:]

# ==== COPY ====
def copy_pair(img: Path, txt: Path, dst_dir: Path):
    dst_dir.mkdir(parents=True, exist_ok=True)
    import shutil as _sh
    _sh.copy2(img, dst_dir / img.name)
    _sh.copy2(txt, dst_dir / txt.name)

for img, txt in val_samples: copy_pair(img, txt, VAL)
for img, txt in train_samples: copy_pair(img, txt, TRAIN)

# ==== REPORT ====
def count_images(d: Path): return sum(1 for f in d.glob("*") if f.suffix.lower() in IMG_EXTS)
def count_txts(d: Path):    return sum(1 for f in d.glob("*.txt"))

print(f"Total pairs found: {n_total}")
print(f"Train pairs: {len(train_samples)} | images={count_images(TRAIN)} | labels={count_txts(TRAIN)}")
print(f"Val   pairs: {len(val_samples)}   | images={count_images(VAL)}   | labels={count_txts(VAL)}")

# ==== WRITE A YOLO DATA YAML ====
yaml_text = f"""# Auto-generated for Colab
path: {BASE}
train: {TRAIN}
val: {VAL}

names:
  0: Crack
  1: ACrack
  2: Efflorescence
  3: WConccor
  4: Spalling
  5: Wetspot
  6: Rust
  7: ExposedRebars
"""
yaml_path = BASE / "data_colab.yaml"
yaml_path.write_text(yaml_text)
print(f"Wrote {yaml_path}")
'''
Path("colab/scripts").mkdir(parents=True, exist_ok=True)
Path("colab/scripts/step5_split_and_yaml.py").write_text(content)
print("✅ Wrote colab/scripts/step5_split_and_yaml.py")


In [ ]:
from pathlib import Path
content = r'''# Polygon-only overlays (no boxes, no shaded masks, no confidences)
import cv2, numpy as np, torch
import torch.nn.functional as F
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.engine.results import Results

def _box_label_noop(self, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)): return
def _masks_noop(self, masks, colors, im_gpu, alpha: float = 0.5, retina_masks: bool = False): return

def _segmentation(self, mask, label='', color=(0, 255, 0), thresh: float = 0.30):
    if isinstance(mask, torch.Tensor):
        m = mask.unsqueeze(0).unsqueeze(0)
        m = F.interpolate(m, size=(self.im.shape[0], self.im.shape[1]), mode='bilinear', align_corners=False)
        m = (m.squeeze().detach().cpu().numpy() > thresh).astype(np.uint8)
    else:
        m = mask
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) < 3: continue
        cv2.polylines(self.im, [cnt], isClosed=True, color=color, thickness=self.lw)
        if label:
            x, y = cnt[0][0]
            cv2.putText(self.im, label, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)

Annotator.box_label = _box_label_noop
Annotator.masks = _masks_noop
Annotator.segmentation = _segmentation

__orig_plot = Results.plot
def _plot_polygons_only(self, conf=True, boxes=True, masks=True, probs=False, labels=True, *args, **kwargs):
    im = (self.orig_img.copy() if getattr(self, "orig_img", None) is not None else self.plot_img.copy())
    annotator = Annotator(im, example=str(self.names))
    pred_masks = getattr(self, "masks", None)
    pred_boxes = getattr(self, "boxes", None)
    if getattr(pred_masks, "data", None) is not None:
        classes = (pred_boxes.cls.tolist() if (pred_boxes is not None and hasattr(pred_boxes, "cls"))
                   else [0] * len(pred_masks.data))
        for m, cls_idx in zip(pred_masks.data, classes):
            cls_idx = int(cls_idx)
            annotator.segmentation(m, label=self.names[cls_idx], color=colors(cls_idx, bgr=True))
    return annotator.result()

Results.plot = _plot_polygons_only
print("✅ Ultralytics patched: polygon-only overlays enabled.")
print("   Restart runtime to undo the patch.")
'''
Path("colab/scripts").mkdir(parents=True, exist_ok=True)
Path("colab/scripts/step8_patch_ultralytics_polygons.py").write_text(content)
print("✅ Wrote colab/scripts/step8_patch_ultralytics_polygons.py")


In [ ]:
from pathlib import Path
content = r'''from ultralytics import YOLO
from pathlib import Path
from collections import Counter
import cv2, torch
import os, json, math, numpy as np

MODEL   = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SOURCE  = "/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test 2"
PROJECT = "/content/drive/MyDrive/yolo_runs"
RUN_NAME = "trial_preds_polygons"
IMGSZ   = 640
CONF    = 0.5
BOX_ALPHA = 0.35

model = YOLO(MODEL)
results = model.predict(source=SOURCE, imgsz=IMGSZ, conf=CONF, save=True, max_det=300, project=PROJECT, name=RUN_NAME)

SAVE_JSON, SAVE_CROPS, PAD_PX = True, True, 8

def to_int_list(x): return [int(round(v)) for v in x]
def poly_area_px2(points_xy):
    if points_xy is None or len(points_xy) < 3: return 0.0
    return float(cv2.contourArea(np.asarray(points_xy, dtype=np.float32)))
def clamp_bbox(x1,y1,x2,y2,w,h):
    x1=max(0,min(int(math.floor(x1)),w-1)); y1=max(0,min(int(math.floor(y1)),h-1))
    x2=max(0,min(int(math.ceil(x2)), w-1)); y2=max(0,min(int(math.ceil(y2)), h-1))
    if x2<=x1:x2=min(x1+1,w-1)
    if y2<=y1:y2=min(y1+1,h-1)
    return x1,y1,x2,y2

def overlay_counts_on_image(img_path: Path, lines, box_alpha=BOX_ALPHA):
    img=cv2.imread(str(img_path))
    if img is None: print(f"⚠️ Could not read image for overlay: {img_path}"); return False
    sizes=[cv2.getTextSize(t, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0][0] for t in lines] or [1]
    box_w=max(sizes)+20; line_h=28; box_h=line_h*max(1,len(lines))+20; x0,y0=5,5
    ov=img.copy(); cv2.rectangle(ov,(x0,y0),(x0+box_w,y0+box_h),(0,0,0),-1)
    img=cv2.addWeighted(ov, box_alpha, img, 1-box_alpha, 0)
    y=y0+20
    for t in lines:
        cv2.putText(img,t,(x0+10,y),cv2.FONT_HERSHEY_SIMPLEX,0.9,(255,255,255),2,cv2.LINE_AA); y+=line_h
    ok=cv2.imwrite(str(img_path),img)
    if not ok: print(f"⚠️ Failed to write overlay to: {img_path}")
    return ok

def find_saved_image(save_dir: Path, src_path: Path):
    direct=save_dir/src_path.name
    if direct.exists(): return direct
    m=list(save_dir.glob(src_path.stem+".*"))
    return m[0] if m else None

if not results: print("No results returned.")
else:
    final_dir = Path(results[0].save_dir); json_dir=final_dir/"json"; crops_dir=final_dir/"crops"
    if SAVE_JSON: json_dir.mkdir(parents=True, exist_ok=True)
    if SAVE_CROPS: crops_dir.mkdir(parents=True, exist_ok=True)
    print("Writing outputs in:", final_dir)

    total_images=0; total_dets=0
    for res in results:
        save_dir=Path(res.save_dir); src_path=Path(res.path)
        out_img=find_saved_image(save_dir, src_path)
        if out_img is None: print(f"⚠️ No saved image for {src_path.name} in {save_dir}"); continue

        names=res.names; dets=[]; cls_idxs=[]; confs=[]; bboxes=[]
        if getattr(res,"boxes",None) is not None:
            if getattr(res.boxes,"cls",None) is not None:
                cls_raw=res.boxes.cls; cls_idxs=(cls_raw.detach().cpu().tolist() if isinstance(cls_raw,torch.Tensor) else list(cls_raw)); cls_idxs=[int(x) for x in cls_idxs]
            if getattr(res.boxes,"conf",None) is not None:
                conf_raw=res.boxes.conf; confs=(conf_raw.detach().cpu().tolist() if isinstance(conf_raw,torch.Tensor) else list(conf_raw))
            if getattr(res.boxes,"xyxy",None) is not None:
                xyxy=res.boxes.xyxy;
                if isinstance(xyxy,torch.Tensor): xyxy=xyxy.detach().cpu().numpy()
                bboxes=xyxy.tolist()

        polys_xy=None
        if getattr(res,"masks",None) is not None and getattr(res.masks,"xy",None) is not None:
            polys_xy=res.masks.xy

        N=len(polys_xy) if polys_xy is not None else len(bboxes)
        img_bgr=cv2.imread(str(src_path)); ih,iw=(img_bgr.shape[0],img_bgr.shape[1]) if img_bgr is not None else (None,None)

        for i in range(N):
            cls_name=names[cls_idxs[i]] if i<len(cls_idxs) else names[0]
            conf=float(confs[i]) if i<len(confs) else None
            if i<len(bboxes):
                x1,y1,x2,y2=bboxes[i]
            else:
                if polys_xy is not None and i<len(polys_xy) and len(polys_xy[i])>=3:
                    px,py=polys_xy[i][:,0],polys_xy[i][:,1]; x1,y1,x2,y2=float(px.min()),float(py.min()),float(px.max()),float(py.max())
                else:
                    x1=y1=0.0; x2=float(iw-1) if iw else 1.0; y2=float(ih-1) if ih else 1.0
            bbox=[int(round(x1)),int(round(y1)),int(round(x2)),int(round(y2))]

            poly_flat=[]; area_px2=None
            if polys_xy is not None and i<len(polys_xy) and len(polys_xy[i])>=3:
                pts=polys_xy[i]; area_px2=poly_area_px2(pts); poly_flat=to_int_list(pts.reshape(-1).tolist())
            else:
                if iw is not None and ih is not None: area_px2=max(1,(bbox[2]-bbox[0])*(bbox[3]-bbox[1]))

            # crops
            crop_rel_path=None
            if SAVE_CROPS and img_bgr is not None:
                x1p,y1p,x2p,y2p=clamp_bbox(bbox[0]-PAD_PX,bbox[1]-PAD_PX,bbox[2]+PAD_PX,bbox[3]+PAD_PX,iw,ih)
                crop=img_bgr[y1p:y2p, x1p:x2p]
                if crop.size>0:
                    cls_dir=crops_dir/cls_name; cls_dir.mkdir(parents=True, exist_ok=True)
                    crop_name=f"{src_path.stem}_{cls_name}_{i:02d}.jpg"
                    cv2.imwrite(str(cls_dir/crop_name), crop)
                    crop_rel_path=str(Path("crops")/cls_name/crop_name)

            dets.append({"class":cls_name,"bbox_xyxy":bbox,"poly_xy_flat":poly_flat,"area_px2":area_px2,"conf":conf,"crop_relpath":crop_rel_path})

        if SAVE_JSON:
            rec={"image_path":str(src_path),"save_path":str(out_img) if out_img else None,"detections":dets}
            (json_dir/f"{src_path.stem}.json").write_text(json.dumps(rec, indent=2))

        counts=Counter([d["class"] for d in dets]) if dets else {}
        lines=[f"{k}: {counts[k]}" for k in sorted(counts.keys())] if counts else ["No detections"]
        (save_dir/f"{src_path.stem}_counts.txt").write_text("\n".join(lines)+"\n")
        if out_img is not None: overlay_counts_on_image(out_img, lines)

        total_images=total_images+1 if 'total_images' in globals() else 1
        total_dets=total_dets+len(dets) if 'total_dets' in globals() else len(dets)
        print(f"✅ {src_path.name}: {len(dets)} detections | counts ->", ", ".join(lines))

    print("\n🎉 Done")
    if SAVE_JSON:  print("   JSON dir :", json_dir)
    if SAVE_CROPS: print("   Crops dir:", crops_dir)
    print(f"   Images processed: {total_images} | Total detections: {total_dets}")
'''
Path("colab/scripts").mkdir(parents=True, exist_ok=True)
Path("colab/scripts/step9_predict_images.py").write_text(content)
print("✅ Wrote colab/scripts/step9_predict_images.py")


In [ ]:
from pathlib import Path
content = r'''from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
import math, torch, gc

MODEL       = "/content/drive/MyDrive/yolo_runs/train_colab_m640/weights/best.pt"
SRC_DIR     = Path("/content/drive/MyDrive/YOLOv11_Concrete-defect-dataset-08202025/trial test videos 2")
PROJECT     = "/content/drive/MyDrive/yolo_runs"
RUN_NAME    = "video_preds_polygons"
CONF        = 0.5

IMGSZ_VID      = 640
VID_STRIDE_VID = 1
IMGSZ_CNT      = 640
VID_STRIDE_CNT = 2
USE_FP16       = True
TRACKER        = "bytetrack.yaml"
PRINT_EVERY    = 50
VIDEO_EXTS     = {".mp4", ".mov", ".avi", ".mkv", ".m4v"}

videos = [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS]
if not videos: raise SystemExit(f"No videos found in {SRC_DIR}")

model = YOLO(MODEL)

print("Saving annotated videos...")
pred_results = model.predict(source=str(SRC_DIR), imgsz=IMGSZ_VID, conf=CONF, save=True, project=PROJECT, name=RUN_NAME, vid_stride=VID_STRIDE_VID, verbose=True)
if not pred_results: raise SystemExit("No prediction results returned.")
run_dir = Path(pred_results[0].save_dir); print(f"\n✅ Videos saved to: {run_dir}")

print("\nCounting unique defects (no video saving)...")
for vf in videos:
    print(f"\n▶ {vf.name}")
    unique_ids_per_class = defaultdict(set)
    names = None; had_ids=False; frame_idx=0

    for res in model.track(source=str(vf), imgsz=IMGSZ_CNT, conf=CONF, tracker=TRACKER, device=0, save=False, stream=True, stream_buffer=False, half=USE_FP16, vid_stride=VID_STRIDE_CNT, verbose=False):
        if names is None: names=res.names
        frame_idx += 1
        if frame_idx % PRINT_EVERY == 0: print(f"  processed frame {frame_idx}")

        boxes = getattr(res, "boxes", None)
        if boxes is None: continue

        cls_t = getattr(boxes, "cls", None)
        id_t  = getattr(boxes, "id",  None)
        if id_t is None or cls_t is None: continue

        had_ids = True
        clses = cls_t.detach().cpu().tolist() if isinstance(cls_t, torch.Tensor) else list(cls_t or [])
        ids   = id_t.detach().cpu().tolist()  if isinstance(id_t,  torch.Tensor) else list(id_t  or [])

        for c, tid in zip(clses, ids):
            if tid is None: continue
            if isinstance(tid, float) and (math.isnan(tid) or tid < 0): continue
            if isinstance(tid, (int, float)) and tid < 0: continue
            unique_ids_per_class[int(c)].add(int(tid))

        del res
        if frame_idx % 100 == 0: torch.cuda.empty_cache(); gc.collect()

    txt_path = run_dir / f"{vf.stem}_counts.txt"
    lines = [f"{names[c]}: {len(s)}" for c, s in sorted(unique_ids_per_class.items()) if len(s)>0] if (had_ids and unique_ids_per_class) else ["No detections"]
    with open(txt_path, "w") as f: f.write("\n".join(lines) + "\n")
    print("  ✅ Wrote counts:", txt_path); [print("   ", L) for L in lines]

print("\n🎬 Done. Videos and *_counts.txt are in:", run_dir)
'''
Path("colab/scripts").mkdir(parents=True, exist_ok=True)
Path("colab/scripts/step10_predict_videos_bytetrack.py").write_text(content)
print("✅ Wrote colab/scripts/step10_predict_videos_bytetrack.py")


In [ ]:
from pathlib import Path

GITHUB_USER = "natnaeltaye"
REPO_NAME = "concrete-defect-yolov11"
BRANCH_NAME = "add-colab-workflow"

colab_md = f"""# Google Colab Workflow

This folder contains a Colab notebook and helper scripts to reproduce training and inference without a VM.

## Open in Colab
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/{GITHUB_USER}/{REPO_NAME}/blob/{BRANCH_NAME}/colab/YOLOv11-Concrete-defects-training-08202025.ipynb)

## Contents
- `YOLOv11-Concrete-defects-training-08202025.ipynb` – end-to-end steps (1–10).
- `scripts/`:
  - `step5_split_and_yaml.py`
  - `step8_patch_ultralytics_polygons.py`
  - `step9_predict_images.py`
  - `step10_predict_videos_bytetrack.py`

## Notes
- Results save to Drive under `/content/drive/MyDrive/yolo_runs`.
- Do not commit large outputs or model weights (`*.pt`, `*.onnx`). Use Zenodo or GitHub Releases and link them.
- Hyperparameters and data YAML are defined in the notebook and scripts for reproducibility.
"""
Path("colab/README.md").write_text(colab_md)

top_add = """
## Reproduction Paths

- **VM workflow**: see `vm/` for training on a virtual machine.
- **Google Colab workflow**: see `colab/` for the end-to-end Colab notebook and helper scripts.
  - One-click: use the Colab badge inside `colab/README.md`.
"""
if Path("README.md").exists():
    with open("README.md","a",encoding="utf-8") as f: f.write(top_add)
else:
    Path("README.md").write_text("# Project\n"+top_add)

gitignore = """
# Large/ephemeral artifacts
*.pt
*.onnx
*.mp4
*.avi
*.mov
*.mkv
*.m4v
*.zip
*.tar
*.tar.gz
*.7z
.DS_Store

# YOLO & Colab outputs
yolo_runs/
runs/
outputs/
*.json
*.csv
*.xlsx
"""
if Path(".gitignore").exists():
    with open(".gitignore","a",encoding="utf-8") as f: f.write("\n"+gitignore.strip()+"\n")
else:
    Path(".gitignore").write_text(gitignore.strip()+"\n")

print("✅ Wrote colab/README.md, updated README.md, and .gitignore")


In [ ]:
!git status
!git add colab/ README.md .gitignore
!git commit -m "Add Colab workflow: separate scripts (Step5, Step8, Step9, Step10) + README + gitignore"
!git push --set-upstream origin add-colab-workflow
print("✅ Branch pushed: add-colab-workflow")


In [ ]:
# Upload your saved notebook file from your computer
import os, shutil
from google.colab import files

print("👉 Choose: YOLOv11-Concrete-defects-training-08202025.ipynb")
uploaded = files.upload()
nb_name = list(uploaded.keys())[0]
os.makedirs("colab", exist_ok=True)
target = "colab/YOLOv11-Concrete-defects-training-08202025.ipynb"
shutil.move(nb_name, target)
print(f"✅ Notebook placed at {target}")


In [ ]:
!git add colab/YOLOv11-Concrete-defects-training-08202025.ipynb
!git commit -m "Add Colab notebook (Steps 1–10)"
!git push
